# [Pandas Pivot Table Explained](http://pbpython.com/pandas-pivot-table-explained.html)

In [ ]:
# install libraries
!pip install pandas openpyxl

In [ ]:
# load libraries
import pandas as pd
import numpy as np

In [ ]:
# load the data
df = pd.read_excel("./sales-funnel.xlsx")

In [ ]:
# first five rows
df.head()

In [ ]:
# examine column data types
df.dtypes

In [ ]:
# indicate that status represents categorical data
df["Status"] = df["Status"].astype("category")

In [ ]:
# set order of categories
df["Status"].cat.set_categories(["won","pending","presented","declined"],inplace=True)

In [ ]:
df.head()

In [ ]:
# sort on status
df.sort_values("Status")

In [ ]:
# create a pivot table with "name" for rows
pd.pivot_table(df, index=["Name"])

In [ ]:
# add fields to the pivot table index
pd.pivot_table(df, index=["Name", "Rep", "Manager"])

In [ ]:
# create a new pivot table with infomration about sales by manager/rep
pd.pivot_table(df, index=["Manager", "Rep"])

In [ ]:
# display only price data
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"])

In [ ]:
# aggregate price data using sum rather than mean
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=np.sum)

In [ ]:
# provide multiple aggregations
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], aggfunc=[np.mean, len])

In [ ]:
# break out price by product
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], 
               columns=['Product'], aggfunc=np.sum)

In [ ]:
# replace "NaN" with zero
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price"], 
               columns=['Product'], aggfunc=np.sum,
               fill_value=0)

In [ ]:
# include both price and quantity data
pd.pivot_table(df, index=["Manager", "Rep"], values=["Price", "Quantity"], 
               columns=['Product'], aggfunc=np.sum,
               fill_value=0)

In [ ]:
# move "Product" from column to rows
pd.pivot_table(df, index=["Manager", "Rep", "Product"], 
               values=["Price", "Quantity"], 
               aggfunc=np.sum, fill_value=0)

In [ ]:
# include totals using "margins" keyword argument
pd.pivot_table(df, index=["Manager", "Rep", "Product"], 
               values=["Price", "Quantity"], 
               aggfunc=np.sum, fill_value=0,
               margins=True)

In [ ]:
# pivot table with manager and status
pd.pivot_table(df, index=["Manager", "Status"], 
               values=["Price", "Quantity"], 
               aggfunc=np.sum, fill_value=0,
               margins=True)

In [ ]:
# piot table with different aggregations for each value
pd.pivot_table(df, index=["Manager","Status"],
               columns=["Product"],
               values=["Quantity","Price"],
               aggfunc={"Quantity":len,"Price":np.sum},
               fill_value=0)

In [ ]:
# use sum for both aggregations
pd.pivot_table(df, index=["Manager","Status"],
               columns=["Product"],
               values=["Quantity","Price"],
               aggfunc={"Quantity":np.sum,"Price":np.sum},
               fill_value=0)

In [ ]:
# assign the pivot table to a variable
table = pd.pivot_table(df, index=["Manager","Status"],
                       columns=["Product"],
                       values=["Quantity","Price"],
                       aggfunc={"Quantity":np.sum,"Price":np.sum},
                       fill_value=0)

In [ ]:
# query the table for data about Debra Henley
table.query('Manager == ["Debra Henley"]')

In [ ]:
# query data only for won or pending statuses
table.query('Status == ["won", "pending"]')

In [ ]:
# show price information for items with won or pending status
table.query('Status == ["won", "pending"]')["Price"]

In [ ]:
# a pivot table is a DataFrame and has dataframe methods like sum
table.sum()

# [Generating Excel Reports](http://pbpython.com/pandas-pivot-report.html)

In [ ]:
# create pivot table with sum of quantity and price aggregrated by manager/rep/product
table = pd.pivot_table(df, index=["Manager", "Rep", "Product"],
                       values=["Quantity", "Price"],
                       fill_value=0)

In [ ]:
# display table
table

In [ ]:
# can query to get information for a specific manager
table.query('Manager == "Debra Henley"')

In [ ]:
# Can use xs (cross section)
table.xs("Debra Henley", level=0)

In [ ]:
# can specify values for each level
table.xs(("Debra Henley", "John Smith"))

In [ ]:
# sepecify value for a specific value of an inner level
table.xs("John Smith", level=1)

In [ ]:
# use table.index.get_level_values() to see the index values at each level associated with each row
# level zero is the outer-most level (manager)
table.index.get_level_values(0)

In [ ]:
# level 1 (rep)
table.index.get_level_values(1)

In [ ]:
# get distinct values using the .unique() method
# unique rep names
table.index.get_level_values(1).unique()

In [ ]:
# unique manager names
table.index.get_level_values(0).unique()

In [ ]:
# use the unique level-0 values to iterate through the pivot table
# use a for-loop to access one manager at a time
for manager in table.index.get_level_values(0).unique():
    print(manager)

In [ ]:
# iterate through table using cross sections
for manager in table.index.get_level_values(0).unique():
    print(table.xs(manager, level=0))

In [ ]:
# cross sections are also DataFrames and DataFrames have a to_excel() method
# to_excel() method requires an ExcelWriter that specifies the file to write to

# iterate through pivot table 
for manager in table.index.get_level_values(0).unique():
    filename = manager + ".xlsx"
    writer = pd.ExcelWriter(filename)  # create an ExcelWriter
    temp_df = table.xs(manager, level=0)
    temp_df.to_excel(writer)  # convert DataFrame to an Excel format
    writer.save()  # save the Excel file

# [Creating HTML Reports](http://pbpython.com/pdf-reports.html)

In [ ]:
# review pivot table
table

In [ ]:
# create an html template with place holders for title and pivot table using multi-line strings
template = """<!DOCTYPE html>
<html>
<head lang="en">
    <meta charset="UTF-8">
    <title>{title}</title>
</head>
<body>
    <h2>{title}</h2>
     {pivot_table_data}
</body>
</html>
"""

In [ ]:
# DataFrames include a .to_html() method
table.to_html()

In [ ]:
# combine template with data
report = template.format(title="Sales Report", pivot_table_data=table.to_html())
report

In [ ]:
# save html to file
with open("report.html", "w") as output:
    output.write(report)